In [2]:
#lastest code'
import cv2
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# --------------------------
# CONFIG: Give folder containing input images
# --------------------------
#input_folder = r"D:\projects\neuro\all fascicles\set2_Image_01_20x_bf_02"
#output_root = r"D:\projects\neuro\all fascicles\set2_Image_01_20x_bf_02_results3"
#os.makedirs(output_root, exist_ok=True)

# Scale factor (µm/px)
scale_factor = 0.136

# --------------------------
# Tunables (UPDATED)
# --------------------------
TISSUE_OVERLAP_MIN = 0.50
MAX_MEAN_INTENSITY = 245

MIN_CIRCULARITY = 0.05        # lower to detect small axons
MIN_CONTOUR_AREA = 8          # reduced from 20
MAX_CONTOUR_AREA = 2000000
MIN_SOLIDITY = 0.15           # reduced from 0.3

# --------------------------
# Helpers
# --------------------------
def darken_and_sharpen(image):
    darkened = cv2.convertScaleAbs(image, alpha=1.5, beta=-20)
    sharpen_kernel = np.array([[0, -1, 0],
                               [-1, 5, -1],
                               [0, -1, 0]])
    return cv2.filter2D(darkened, -1, sharpen_kernel)

def build_tissue_mask(full_image):
    hsv = cv2.cvtColor(full_image, cv2.COLOR_BGR2HSV)
    H, S, V = cv2.split(hsv)
    sat_mask = cv2.inRange(S, 15, 255)
    not_too_bright = cv2.inRange(V, 0, 250)

    mask = cv2.bitwise_and(sat_mask, not_too_bright)
    k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))

    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, k, iterations=1)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, k, iterations=2)
    return mask

def contour_overlap_ratio(contour, mask):
    c_mask = np.zeros(mask.shape, dtype=np.uint8)
    cv2.drawContours(c_mask, [contour], -1, 255, -1)
    inter = cv2.bitwise_and(c_mask, mask)
    area_c = cv2.countNonZero(c_mask)
    area_i = cv2.countNonZero(inter)
    return area_i / float(area_c) if area_c > 0 else 0.0

def mean_intensity_in_contour(gray, contour):
    m = np.zeros(gray.shape, dtype=np.uint8)
    cv2.drawContours(m, [contour], -1, 255, -1)
    return cv2.mean(gray, mask=m)[0]

def circularity(contour):
    a = cv2.contourArea(contour)
    p = cv2.arcLength(contour, True)
    return 4 * np.pi * a / (p * p + 1e-6) if p > 0 else 0

# --------------------------
# Main patch processing
# --------------------------
def process_patch(patch, mask_patch, x_offset, y_offset, axon_data, object_counter, output_image):

    patch = darken_and_sharpen(patch)
    gray = cv2.cvtColor(patch, cv2.COLOR_BGR2GRAY)

    # Small smoothing for better boundary separation
    smooth = cv2.bilateralFilter(gray, 5, 20, 20)

    # CLAHE
    clahe = cv2.createCLAHE(clipLimit=2.5, tileGridSize=(8, 8))
    enhanced = clahe.apply(smooth)

    # Thresholding
    blurred = cv2.GaussianBlur(enhanced, (3, 3), 0)
    _, otsu = cv2.threshold(blurred, 0, 255,
                            cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    adaptive = cv2.adaptiveThreshold(
        blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY_INV, 35, 2
    )

    thresh = cv2.bitwise_or(otsu, adaptive)

    # ---------------------------
    # NEW: Separate touching axons using WATERSHED
    # ---------------------------
    # thresh: foreground (axon structures) = 255, background = 0
    kernel = np.ones((3, 3), np.uint8)

    # Remove tiny noise, keep main blobs
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=1)

    # Sure background
    sure_bg = cv2.dilate(opening, kernel, iterations=3)

    # Sure foreground via distance transform
    dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
    if dist_transform.max() > 0:
        _, sure_fg = cv2.threshold(dist_transform,
                                   0.4 * dist_transform.max(), 255, 0)
    else:
        sure_fg = np.zeros_like(opening)
    sure_fg = np.uint8(sure_fg)

    # Unknown region (possible boundaries between axons)
    unknown = cv2.subtract(sure_bg, sure_fg)

    # Marker labelling for watershed
    num_markers, markers = cv2.connectedComponents(sure_fg)
    markers = markers + 1  # background becomes 1 instead of 0
    markers[unknown == 255] = 0

    # Watershed needs 3-channel image
    wshed_input = cv2.cvtColor(enhanced, cv2.COLOR_GRAY2BGR)
    markers = cv2.watershed(wshed_input, markers)

    # Use watershed boundaries to carve gaps between touching axons
    cleaned = thresh.copy()
    cleaned[markers == -1] = 0  # make watershed boundary black (gap)
    # ---------------------------

    contours, hierarchy = cv2.findContours(
        cleaned, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE
    )
    if hierarchy is None:
        return object_counter
    hierarchy = hierarchy[0]

    for i, contour in enumerate(contours):

        # Only consider outermost contours
        if hierarchy[i][3] != -1:
            continue

        area = cv2.contourArea(contour)
        if area < MIN_CONTOUR_AREA or area > MAX_CONTOUR_AREA:
            continue

        overlap = contour_overlap_ratio(contour, mask_patch)
        if overlap < TISSUE_OVERLAP_MIN:
            continue

        hull = cv2.convexHull(contour)
        solidity = area / (cv2.contourArea(hull) + 1e-6)
        circ = circularity(contour)
        if solidity < MIN_SOLIDITY or circ < MIN_CIRCULARITY:
            continue

        # inner contours
        inner_children = []
        for j, child in enumerate(contours):
            if hierarchy[j][3] == i and cv2.contourArea(child) > 30:
                inner_children.append(child)

        if len(inner_children) == 0:
            continue

        axon_type = "mature" if len(inner_children) <= 2 else "regrowth_cluster"

        (x_outer, y_outer), outer_radius = cv2.minEnclosingCircle(contour)
        contour_shifted = contour + np.array([x_offset, y_offset])
        x_outer += x_offset
        y_outer += y_offset

        outer_area_px = cv2.contourArea(contour)
        outer_area_um2 = outer_area_px * (scale_factor ** 2)

        outer_color = (255, 0, 0) if axon_type == "mature" else (0, 255, 255)
        cv2.drawContours(output_image, [contour_shifted], -1, outer_color, 1)

        inner_radii = []
        inner_areas = []

        for inner_c in inner_children:
            (x_inner, y_inner), inner_radius = cv2.minEnclosingCircle(inner_c)
            inner_shifted = inner_c + np.array([x_offset, y_offset])
            cv2.drawContours(output_image, [inner_shifted], -1, (0, 255, 0), 1)
            inner_radii.append(inner_radius)
            inner_areas.append(cv2.contourArea(inner_c))

        largest_inner = inner_children[np.argmax(inner_areas)]
        inner_radius = max(inner_radii)
        inner_area_px = max(inner_areas)
        inner_area_um2 = inner_area_px * (scale_factor ** 2)

        thickness = (outer_radius - inner_radius) * scale_factor
        g_ratio = inner_radius / (outer_radius + 1e-6)
        area_ratio = inner_area_um2 / (outer_area_um2 + 1e-6)

        axon_data.append({
            "axon_id": object_counter,
            "axon_type": axon_type,
            "num_inner_contours": len(inner_children),
            "center_x": x_outer,
            "center_y": y_outer,
            "outer_radius": outer_radius * scale_factor,
            "inner_radius": inner_radius * scale_factor,
            "thickness": thickness,
            "diameter": (2 * outer_radius) * scale_factor,
            "outer_area": outer_area_um2,
            "inner_area": inner_area_um2,
            "area_ratio": area_ratio,
            "hole_ratio": inner_area_um2 / (outer_area_um2 + 1e-6),
            "eccentricity": 0,
            "g_ratio": g_ratio
        })

        cv2.putText(output_image, str(object_counter),
                    (int(x_outer), int(y_outer)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

        object_counter += 1

    return object_counter

# --------------------------
# Whole image processing
# --------------------------
def process_image(image_path, patch_size=1024):
    image = cv2.imread(image_path)
    if image is None:
        print(f" Cannot read image: {image_path}")
        return None, None

    tissue_mask_full = build_tissue_mask(image)
    h, w = image.shape[:2]

    output_image = image.copy()
    axon_data = []
    object_counter = 1

    for y in range(0, h, patch_size):
        for x in range(0, w, patch_size):
            patch = image[y:y+patch_size, x:x+patch_size]
            mask_patch = tissue_mask_full[y:y+patch_size, x:x+patch_size]

            object_counter = process_patch(
                patch, mask_patch, x, y, axon_data, object_counter, output_image
            )

    print(f"Processed {len(axon_data)} axons.")
    return axon_data, output_image

# --------------------------
# Save CSV
# --------------------------
def save_to_csv(axon_data, output_folder):
    df = pd.DataFrame(axon_data)
    csv_path = os.path.join(output_folder, "axon_measurements.csv")
    df.to_csv(csv_path, index=False)
    return df

# --------------------------
# Visualizations
# --------------------------
def plot_and_visualize(df, output_folder):
    numeric_cols = df.select_dtypes(include=[np.number]).columns

    for col in numeric_cols:
        vals = df[col].dropna()
        if len(vals) == 0:
            continue

        plt.hist(vals, bins=25)
        plt.title(col)
        plt.savefig(os.path.join(output_folder, f"hist_{col}.png"))
        plt.close()

# --------------------------
# Process entire folder
# --------------------------
def process_folder(input_folder, output_root):
    images = [
        f for f in os.listdir(input_folder)
        if f.lower().endswith((".png", ".jpg", ".jpeg", ".bmp"))
    ]

    for img_name in images:
        print("\n==============================")
        print(f"Processing {img_name}")
        print("==============================")

        img_path = os.path.join(input_folder, img_name)
        img_name_wo_ext = os.path.splitext(img_name)[0]
        img_output_folder = os.path.join(output_root, img_name_wo_ext)
        os.makedirs(img_output_folder, exist_ok=True)

        axon_data, output_image = process_image(img_path)

        if axon_data is None:
            continue

        df = save_to_csv(axon_data, img_output_folder)
        plot_and_visualize(df, img_output_folder)
        image = cv2.imread(img_path)
        tissue_mask = build_tissue_mask(image)
        
        mask_path = os.path.join(img_output_folder, "tissue_mask.png")
        cv2.imwrite(mask_path, tissue_mask)

        out_img_path = os.path.join(img_output_folder, f"numbered_{img_name}")
        cv2.imwrite(out_img_path, output_image)

        print(f" Saved results for {img_name} in {img_output_folder}\n")

def process_multiple_folders(input_root, output_root):
    subfolders = [
        f for f in os.listdir(input_root)
        if os.path.isdir(os.path.join(input_root, f))
    ]

    print(f"Found {len(subfolders)} folders:", subfolders)

    for folder_name in subfolders:
        input_subfolder = os.path.join(input_root, folder_name)
        output_subfolder = os.path.join(output_root, folder_name)

        os.makedirs(output_subfolder, exist_ok=True)

        print("\n====================================")
        print(f"Processing folder: {folder_name}")
        print("====================================")

        process_folder(input_subfolder, output_subfolder)

    print("\n ALL FOLDERS PROCESSED!")


# --------------------------
# RUN
# --------------------------
input_root = r"D:\projects\neuro\all fascicles\fascicle images"
output_root = r"D:\projects\neuro\all fascicles_results"

process_multiple_folders(input_root, output_root)


print("\n DONE! All images processed.")


Found 9 folders: ['set1_Image02_40x_bf_03_fascicle_All', 'set1_image03_40x_bf_12_fascicle_All', 'set1_image04_40x_bf_06_fascicle_All', 'set1_image05_40x_bf_02_all', 'set1_Image_01_40x_bf_02', 'set2_Image_01_20x_bf_02', 'set2_Image_01_20x_bf_05_all', 'set2_Image_02_20x_bf_03', 'set2_image_20x_bf_10_all']

Processing folder: set1_Image02_40x_bf_03_fascicle_All

Processing 1.png
Processed 440 axons.
✔ Saved results for 1.png in D:\projects\neuro\all fascicles_results\set1_Image02_40x_bf_03_fascicle_All\1


Processing 2.png
Processed 374 axons.
✔ Saved results for 2.png in D:\projects\neuro\all fascicles_results\set1_Image02_40x_bf_03_fascicle_All\2


Processing 3.png
Processed 202 axons.
✔ Saved results for 3.png in D:\projects\neuro\all fascicles_results\set1_Image02_40x_bf_03_fascicle_All\3


Processing 4.png
Processed 274 axons.
✔ Saved results for 4.png in D:\projects\neuro\all fascicles_results\set1_Image02_40x_bf_03_fascicle_All\4


Processing 5.png
Processed 409 axons.
✔ Saved resu

In [9]:
#lastest code'
import cv2
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# --------------------------
# CONFIG: Give folder containing input images
# --------------------------
#input_folder = r"D:\projects\neuro\all fascicles\set2_Image_01_20x_bf_02"
#output_root = r"D:\projects\neuro\all fascicles\set2_Image_01_20x_bf_02_results3"
#os.makedirs(output_root, exist_ok=True)

# Scale factor (µm/px)
scale_factor = 0.136

# --------------------------
# Tunables (UPDATED)
# --------------------------
TISSUE_OVERLAP_MIN = 0.50
MAX_MEAN_INTENSITY = 245

MIN_CIRCULARITY = 0.05        # lower to detect small axons
MIN_CONTOUR_AREA = 8          # reduced from 20
MAX_CONTOUR_AREA = 2000000
MIN_SOLIDITY = 0.15           # reduced from 0.3

# --------------------------
# Helpers
# --------------------------
def darken_and_sharpen(image):
    darkened = cv2.convertScaleAbs(image, alpha=1.5, beta=-20)
    sharpen_kernel = np.array([[0, -1, 0],
                               [-1, 5, -1],
                               [0, -1, 0]])
    return cv2.filter2D(darkened, -1, sharpen_kernel)

def build_tissue_mask(full_image):
    hsv = cv2.cvtColor(full_image, cv2.COLOR_BGR2HSV)
    H, S, V = cv2.split(hsv)
    sat_mask = cv2.inRange(S, 15, 255)
    not_too_bright = cv2.inRange(V, 0, 250)

    mask = cv2.bitwise_and(sat_mask, not_too_bright)
    k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))

    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, k, iterations=1)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, k, iterations=2)
    return mask

def contour_overlap_ratio(contour, mask):
    c_mask = np.zeros(mask.shape, dtype=np.uint8)
    cv2.drawContours(c_mask, [contour], -1, 255, -1)
    inter = cv2.bitwise_and(c_mask, mask)
    area_c = cv2.countNonZero(c_mask)
    area_i = cv2.countNonZero(inter)
    return area_i / float(area_c) if area_c > 0 else 0.0

def mean_intensity_in_contour(gray, contour):
    m = np.zeros(gray.shape, dtype=np.uint8)
    cv2.drawContours(m, [contour], -1, 255, -1)
    return cv2.mean(gray, mask=m)[0]

def circularity(contour):
    a = cv2.contourArea(contour)
    p = cv2.arcLength(contour, True)
    return 4 * np.pi * a / (p * p + 1e-6) if p > 0 else 0

# --------------------------
# Wiener deconvultion
# --------------------------

def wiener_deconvolution(gray, kernel_size=7, sigma=2.0, K=0.01):
    """
    Safe Wiener deconvolution that adapts PSF to patch size
    """

    h, w = gray.shape

    # If patch is too small, skip deblurring
    if h < kernel_size or w < kernel_size:
        return gray.copy()

    gray_f = gray.astype(np.float32)

    # Ensure kernel size fits patch
    k = min(kernel_size, h, w)
    if k % 2 == 0:
        k -= 1
    k = max(k, 3)

    # Build Gaussian PSF
    psf_1d = cv2.getGaussianKernel(k, sigma)
    psf = psf_1d @ psf_1d.T
    psf /= psf.sum()

    # Pad PSF to image size
    psf_padded = np.zeros_like(gray_f)
    kh, kw = psf.shape
    psf_padded[:kh, :kw] = psf
    psf_padded = np.roll(psf_padded, -kh // 2, axis=0)
    psf_padded = np.roll(psf_padded, -kw // 2, axis=1)

    # FFT
    G = np.fft.fft2(gray_f)
    H = np.fft.fft2(psf_padded)

    # Wiener filter
    H_conj = np.conj(H)
    denom = np.abs(H) ** 2 + K
    F_hat = (H_conj / denom) * G

    # Inverse FFT
    deconv = np.fft.ifft2(F_hat)
    deconv = np.real(deconv)

    # Normalize
    deconv = cv2.normalize(deconv, None, 0, 255, cv2.NORM_MINMAX)
    return deconv.astype(np.uint8)


# --------------------------
# Main patch processing
# --------------------------
def process_patch(patch, mask_patch, x_offset, y_offset,
                  axon_data, object_counter, output_image,
                  use_deblur=False):
    gray = cv2.cvtColor(patch, cv2.COLOR_BGR2GRAY)

    if use_deblur:
        gray = wiener_deconvolution(
            gray,
            kernel_size=7,
            sigma=2.0,
            K=0.01
        )
    
    # Convert back to BGR for rest of pipeline
    patch = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)
    patch = darken_and_sharpen(patch)
    gray = cv2.cvtColor(patch, cv2.COLOR_BGR2GRAY)


    # Small smoothing for better boundary separation
    smooth = cv2.bilateralFilter(gray, 5, 20, 20)

    # CLAHE
    clahe = cv2.createCLAHE(clipLimit=2.5, tileGridSize=(8, 8))
    enhanced = clahe.apply(smooth)

    # Thresholding
    blurred = cv2.GaussianBlur(enhanced, (3, 3), 0)
    _, otsu = cv2.threshold(blurred, 0, 255,
                            cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    adaptive = cv2.adaptiveThreshold(
        blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY_INV, 35, 2
    )

    thresh = cv2.bitwise_or(otsu, adaptive)

    # ---------------------------
    # NEW: Separate touching axons using WATERSHED
    # ---------------------------
    # thresh: foreground (axon structures) = 255, background = 0
    kernel = np.ones((3, 3), np.uint8)

    # Remove tiny noise, keep main blobs
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=1)

    # Sure background
    sure_bg = cv2.dilate(opening, kernel, iterations=3)

    # Sure foreground via distance transform
    dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
    if dist_transform.max() > 0:
        _, sure_fg = cv2.threshold(dist_transform,
                                   0.4 * dist_transform.max(), 255, 0)
    else:
        sure_fg = np.zeros_like(opening)
    sure_fg = np.uint8(sure_fg)

    # Unknown region (possible boundaries between axons)
    unknown = cv2.subtract(sure_bg, sure_fg)

    # Marker labelling for watershed
    num_markers, markers = cv2.connectedComponents(sure_fg)
    markers = markers + 1  # background becomes 1 instead of 0
    markers[unknown == 255] = 0

    # Watershed needs 3-channel image
    wshed_input = cv2.cvtColor(enhanced, cv2.COLOR_GRAY2BGR)
    markers = cv2.watershed(wshed_input, markers)

    # Use watershed boundaries to carve gaps between touching axons
    cleaned = thresh.copy()
    cleaned[markers == -1] = 0  # make watershed boundary black (gap)
    # ---------------------------

    contours, hierarchy = cv2.findContours(
        cleaned, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE
    )
    if hierarchy is None:
        return object_counter
    hierarchy = hierarchy[0]

    for i, contour in enumerate(contours):

        # Only consider outermost contours
        if hierarchy[i][3] != -1:
            continue

        area = cv2.contourArea(contour)
        if area < MIN_CONTOUR_AREA or area > MAX_CONTOUR_AREA:
            continue

        overlap = contour_overlap_ratio(contour, mask_patch)
        if overlap < TISSUE_OVERLAP_MIN:
            continue

        hull = cv2.convexHull(contour)
        solidity = area / (cv2.contourArea(hull) + 1e-6)
        circ = circularity(contour)
        if solidity < MIN_SOLIDITY or circ < MIN_CIRCULARITY:
            continue

        # inner contours
        inner_children = []
        for j, child in enumerate(contours):
            if hierarchy[j][3] == i and cv2.contourArea(child) > 30:
                inner_children.append(child)

        if len(inner_children) == 0:
            continue

        axon_type = "mature" if len(inner_children) <= 2 else "regrowth_cluster"

        (x_outer, y_outer), outer_radius = cv2.minEnclosingCircle(contour)
        contour_shifted = contour + np.array([x_offset, y_offset])
        x_outer += x_offset
        y_outer += y_offset

        outer_area_px = cv2.contourArea(contour)
        outer_area_um2 = outer_area_px * (scale_factor ** 2)

        outer_color = (255, 0, 0) if axon_type == "mature" else (0, 255, 255)
        cv2.drawContours(output_image, [contour_shifted], -1, outer_color, 1)

        inner_radii = []
        inner_areas = []

        for inner_c in inner_children:
            (x_inner, y_inner), inner_radius = cv2.minEnclosingCircle(inner_c)
            inner_shifted = inner_c + np.array([x_offset, y_offset])
            cv2.drawContours(output_image, [inner_shifted], -1, (0, 255, 0), 1)
            inner_radii.append(inner_radius)
            inner_areas.append(cv2.contourArea(inner_c))

        largest_inner = inner_children[np.argmax(inner_areas)]
        inner_radius = max(inner_radii)
        inner_area_px = max(inner_areas)
        inner_area_um2 = inner_area_px * (scale_factor ** 2)

        thickness = (outer_radius - inner_radius) * scale_factor
        g_ratio = inner_radius / (outer_radius + 1e-6)
        area_ratio = inner_area_um2 / (outer_area_um2 + 1e-6)

        axon_data.append({
            "axon_id": object_counter,
            "axon_type": axon_type,
            "num_inner_contours": len(inner_children),
            "center_x": x_outer,
            "center_y": y_outer,
            "outer_radius": outer_radius * scale_factor,
            "inner_radius": inner_radius * scale_factor,
            "thickness": thickness,
            "diameter": (2 * outer_radius) * scale_factor,
            "outer_area": outer_area_um2,
            "inner_area": inner_area_um2,
            "area_ratio": area_ratio,
            "hole_ratio": inner_area_um2 / (outer_area_um2 + 1e-6),
            "eccentricity": 0,
            "g_ratio": g_ratio
        })

        cv2.putText(output_image, str(object_counter),
                    (int(x_outer), int(y_outer)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

        object_counter += 1

    return object_counter

# --------------------------
# Whole image processing
# --------------------------
def process_image(image_path, patch_size=1024):
    image = cv2.imread(image_path)
    if image is None:
        print(f" Cannot read image: {image_path}")
        return None, None

    tissue_mask_full = build_tissue_mask(image)
    h, w = image.shape[:2]

    results = {}

    for mode in ["nodeblur", "deblur"]:
        output_image = image.copy()
        axon_data = []
        object_counter = 1
    
        use_deblur = (mode == "deblur")
    
        for y in range(0, h, patch_size):
            for x in range(0, w, patch_size):
                patch = image[y:y+patch_size, x:x+patch_size]
                mask_patch = tissue_mask_full[y:y+patch_size, x:x+patch_size]
    
                object_counter = process_patch(
                    patch, mask_patch, x, y,
                    axon_data, object_counter,
                    output_image,
                    use_deblur=use_deblur
                )
    
        results[mode] = (axon_data, output_image)
    
    return results


    # for y in range(0, h, patch_size):
    #     for x in range(0, w, patch_size):
    #         patch = image[y:y+patch_size, x:x+patch_size]
    #         mask_patch = tissue_mask_full[y:y+patch_size, x:x+patch_size]

    #         object_counter = process_patch(
    #             patch, mask_patch, x, y, axon_data, object_counter, output_image
    #         )

    # print(f"Processed {len(axon_data)} axons.")
    # return axon_data, output_image

# --------------------------
# Save CSV
# --------------------------
def save_to_csv(axon_data, output_folder):
    df = pd.DataFrame(axon_data)
    csv_path = os.path.join(output_folder, "axon_measurements.csv")
    df.to_csv(csv_path, index=False)
    return df

# --------------------------
# Visualizations
# --------------------------
def plot_and_visualize(df, output_folder):
    numeric_cols = df.select_dtypes(include=[np.number]).columns

    for col in numeric_cols:
        vals = df[col].dropna()
        if len(vals) == 0:
            continue

        plt.hist(vals, bins=25)
        plt.title(col)
        plt.savefig(os.path.join(output_folder, f"hist_{col}.png"))
        plt.close()

# --------------------------
# for viewing effects of blur
# --------------------------
def generate_deblur_preview(image, tissue_mask,
                            kernel_size=7, sigma=2.0, K=0.03):
    """
    Generates full-image Wiener deblur preview (tissue-only)
    """
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Mask background BEFORE deblurring
    gray_masked = gray.copy()
    gray_masked[tissue_mask == 0] = 0

    # Apply Wiener
    deblurred = wiener_deconvolution(
        gray_masked,
        kernel_size=kernel_size,
        sigma=sigma,
        K=K
    )

    return gray, deblurred

# --------------------------
# Process entire folder
# --------------------------
def process_folder(input_folder, output_root):
    images = [
        f for f in os.listdir(input_folder)
        if f.lower().endswith((".png", ".jpg", ".jpeg", ".bmp"))
    ]

    for img_name in images:
        print("\n==============================")
        print(f"Processing {img_name}")
        print("==============================")

        img_path = os.path.join(input_folder, img_name)
        img_name_wo_ext = os.path.splitext(img_name)[0]
        img_output_folder = os.path.join(output_root, img_name_wo_ext)
        os.makedirs(img_output_folder, exist_ok=True)

        #  Run pipeline ONCE
        results = process_image(img_path)

        #  Save nodeblur & deblur outputs
        for mode, (axon_data, output_image) in results.items():
            mode_folder = os.path.join(img_output_folder, mode)
            os.makedirs(mode_folder, exist_ok=True)

            df = save_to_csv(axon_data, mode_folder)
            plot_and_visualize(df, mode_folder)

            out_img_path = os.path.join(mode_folder, f"numbered_{img_name}")
            cv2.imwrite(out_img_path, output_image)

        # 🔹 Save tissue mask ONCE (shared)
        #image = cv2.imread(img_path)
        #tissue_mask = build_tissue_mask(image)
        # cv2.imwrite(os.path.join(img_output_folder, "tissue_mask.png"), tissue_mask)
        # -------------------------
        # Save deblur preview images
        # -------------------------
        image = cv2.imread(img_path)
        tissue_mask = build_tissue_mask(image)
        
        orig_gray, deblur_gray = generate_deblur_preview(
            image,
            tissue_mask,
            kernel_size=7,
            sigma=2.0,
            K=0.03
        )
        
        # Save originals
        cv2.imwrite(os.path.join(img_output_folder, "input_gray.png"), orig_gray)
        cv2.imwrite(os.path.join(img_output_folder, "deblur_gray.png"), deblur_gray)
        
        # Optional: difference image (what Wiener changed)
        diff = cv2.absdiff(orig_gray, deblur_gray)
        cv2.imwrite(os.path.join(img_output_folder, "deblur_difference.png"), diff)
        
        # Save tissue mask (already doing this)
        cv2.imwrite(os.path.join(img_output_folder, "tissue_mask.png"), tissue_mask)




        # if axon_data is None:
        #     continue

        # df = save_to_csv(axon_data, img_output_folder)
        # plot_and_visualize(df, img_output_folder)
        # image = cv2.imread(img_path)
        # tissue_mask = build_tissue_mask(image)
        
        # mask_path = os.path.join(img_output_folder, "tissue_mask.png")
        # cv2.imwrite(mask_path, tissue_mask)

        # out_img_path = os.path.join(img_output_folder, f"numbered_{img_name}")
        # cv2.imwrite(out_img_path, output_image)

        # print(f"✔ Saved results for {img_name} in {img_output_folder}\n")

def process_multiple_folders(input_root, output_root):
    subfolders = [
        f for f in os.listdir(input_root)
        if os.path.isdir(os.path.join(input_root, f))
    ]

    print(f"Found {len(subfolders)} folders:", subfolders)

    for folder_name in subfolders:
        input_subfolder = os.path.join(input_root, folder_name)
        output_subfolder = os.path.join(output_root, folder_name)

        os.makedirs(output_subfolder, exist_ok=True)

        print("\n====================================")
        print(f"Processing folder: {folder_name}")
        print("====================================")

        process_folder(input_subfolder, output_subfolder)

    print("\n ALL FOLDERS PROCESSED!")


# --------------------------
# RUN
# --------------------------
input_root = r"D:\projects\neuro\all fascicles\fascicle images"
output_root = r"D:\projects\neuro\all fascicles_results_deblur_wiener"

process_multiple_folders(input_root, output_root)


print("\n DONE! All images processed.")


Found 9 folders: ['set1_Image02_40x_bf_03_fascicle_All', 'set1_image03_40x_bf_12_fascicle_All', 'set1_image04_40x_bf_06_fascicle_All', 'set1_image05_40x_bf_02_all', 'set1_Image_01_40x_bf_02', 'set2_Image_01_20x_bf_02', 'set2_Image_01_20x_bf_05_all', 'set2_Image_02_20x_bf_03', 'set2_image_20x_bf_10_all']

Processing folder: set1_Image02_40x_bf_03_fascicle_All

Processing 1.png

Processing 2.png

Processing 3.png

Processing 4.png

Processing 5.png

Processing 6.png

Processing 7.png

Processing 8.png

Processing 9.png

Processing folder: set1_image03_40x_bf_12_fascicle_All

Processing 1.png

Processing 10.png

Processing 2.png

Processing 3.png

Processing 4.png

Processing 5.png

Processing 6.png

Processing 7.png

Processing 8.png

Processing 9.png

Processing folder: set1_image04_40x_bf_06_fascicle_All

Processing 1.png

Processing 10.png

Processing 11.png

Processing 2.png

Processing 3.png

Processing 4.png

Processing 5.png

Processing 6.png

Processing 7.png

Processing 8.png

P

In [10]:
#lastest code'
import cv2
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# --------------------------
# CONFIG: Give folder containing input images
# --------------------------
#input_folder = r"D:\projects\neuro\all fascicles\set2_Image_01_20x_bf_02"
#output_root = r"D:\projects\neuro\all fascicles\set2_Image_01_20x_bf_02_results3"
#os.makedirs(output_root, exist_ok=True)

# Scale factor (µm/px)
scale_factor = 0.136

# --------------------------
# Tunables (UPDATED)
# --------------------------
TISSUE_OVERLAP_MIN = 0.50
MAX_MEAN_INTENSITY = 245

MIN_CIRCULARITY = 0.05        # lower to detect small axons
MIN_CONTOUR_AREA = 8          # reduced from 20
MAX_CONTOUR_AREA = 2000000
MIN_SOLIDITY = 0.15           # reduced from 0.3

# --------------------------
# Helpers
# --------------------------
def darken_and_sharpen(image):
    darkened = cv2.convertScaleAbs(image, alpha=1.5, beta=-20)
    sharpen_kernel = np.array([[0, -1, 0],
                               [-1, 5, -1],
                               [0, -1, 0]])
    return cv2.filter2D(darkened, -1, sharpen_kernel)

def build_tissue_mask(full_image):
    hsv = cv2.cvtColor(full_image, cv2.COLOR_BGR2HSV)
    H, S, V = cv2.split(hsv)
    sat_mask = cv2.inRange(S, 15, 255)
    not_too_bright = cv2.inRange(V, 0, 250)

    mask = cv2.bitwise_and(sat_mask, not_too_bright)
    k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))

    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, k, iterations=1)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, k, iterations=2)
    return mask

def contour_overlap_ratio(contour, mask):
    c_mask = np.zeros(mask.shape, dtype=np.uint8)
    cv2.drawContours(c_mask, [contour], -1, 255, -1)
    inter = cv2.bitwise_and(c_mask, mask)
    area_c = cv2.countNonZero(c_mask)
    area_i = cv2.countNonZero(inter)
    return area_i / float(area_c) if area_c > 0 else 0.0

def mean_intensity_in_contour(gray, contour):
    m = np.zeros(gray.shape, dtype=np.uint8)
    cv2.drawContours(m, [contour], -1, 255, -1)
    return cv2.mean(gray, mask=m)[0]

def circularity(contour):
    a = cv2.contourArea(contour)
    p = cv2.arcLength(contour, True)
    return 4 * np.pi * a / (p * p + 1e-6) if p > 0 else 0

# --------------------------
# Wiener deconvultion
# --------------------------

def wiener_deconvolution(gray, kernel_size=7, sigma=2.0, K=0.01):
    """
    Safe Wiener deconvolution that adapts PSF to patch size
    """

    h, w = gray.shape

    # If patch is too small, skip deblurring
    if h < kernel_size or w < kernel_size:
        return gray.copy()

    gray_f = gray.astype(np.float32)

    # Ensure kernel size fits patch
    k = min(kernel_size, h, w)
    if k % 2 == 0:
        k -= 1
    k = max(k, 3)

    # Build Gaussian PSF
    psf_1d = cv2.getGaussianKernel(k, sigma)
    psf = psf_1d @ psf_1d.T
    psf /= psf.sum()

    # Pad PSF to image size
    psf_padded = np.zeros_like(gray_f)
    kh, kw = psf.shape
    psf_padded[:kh, :kw] = psf
    psf_padded = np.roll(psf_padded, -kh // 2, axis=0)
    psf_padded = np.roll(psf_padded, -kw // 2, axis=1)

    # FFT
    G = np.fft.fft2(gray_f)
    H = np.fft.fft2(psf_padded)

    # Wiener filter
    H_conj = np.conj(H)
    denom = np.abs(H) ** 2 + K
    F_hat = (H_conj / denom) * G

    # Inverse FFT
    deconv = np.fft.ifft2(F_hat)
    deconv = np.real(deconv)

    # Normalize
    deconv = cv2.normalize(deconv, None, 0, 255, cv2.NORM_MINMAX)
    return deconv.astype(np.uint8)


# --------------------------
# Main patch processing
# --------------------------
def process_patch(patch, mask_patch, x_offset, y_offset,
                  axon_data, object_counter, output_image,
                  use_deblur=False):

    # Convert to grayscale
    gray = cv2.cvtColor(patch, cv2.COLOR_BGR2GRAY)

    if use_deblur:
        # -----------------------------
        # APPLY TISSUE MASK BEFORE DEBLUR
        # -----------------------------
        masked_gray = gray.copy()
        masked_gray[mask_patch == 0] = 0   # zero out background

        # Wiener deconvolution ONLY on tissue
        masked_gray = wiener_deconvolution(
            masked_gray,
            kernel_size=7,
            sigma=2.0,
            K=0.03      # slightly higher regularization (important)
        )

        # Keep background suppressed
        gray = masked_gray

    # Convert back to BGR for rest of pipeline
    patch = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)
    patch = darken_and_sharpen(patch)
    gray = cv2.cvtColor(patch, cv2.COLOR_BGR2GRAY)


    # Small smoothing for better boundary separation
    smooth = cv2.bilateralFilter(gray, 5, 20, 20)
    
    # Contrast enhancement (conditional)
    if use_deblur:
        # IMPORTANT: skip CLAHE after deblurring
        enhanced = smooth
    else:
        clahe = cv2.createCLAHE(clipLimit=2.5, tileGridSize=(8, 8))
        enhanced = clahe.apply(smooth)
    

    # Thresholding
    blurred = cv2.GaussianBlur(enhanced, (3, 3), 0)
    _, otsu = cv2.threshold(blurred, 0, 255,
                            cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    adaptive = cv2.adaptiveThreshold(
        blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY_INV, 35, 2
    )

    thresh = cv2.bitwise_or(otsu, adaptive)

    # ---------------------------
    # NEW: Separate touching axons using WATERSHED
    # ---------------------------
    # thresh: foreground (axon structures) = 255, background = 0
    kernel = np.ones((3, 3), np.uint8)

    # Remove tiny noise, keep main blobs
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=1)

    # Sure background
    sure_bg = cv2.dilate(opening, kernel, iterations=3)

    # Sure foreground via distance transform
    dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
    if dist_transform.max() > 0:
        _, sure_fg = cv2.threshold(dist_transform,
                                   0.4 * dist_transform.max(), 255, 0)
    else:
        sure_fg = np.zeros_like(opening)
    sure_fg = np.uint8(sure_fg)

    # Unknown region (possible boundaries between axons)
    unknown = cv2.subtract(sure_bg, sure_fg)

    # Marker labelling for watershed
    num_markers, markers = cv2.connectedComponents(sure_fg)
    markers = markers + 1  # background becomes 1 instead of 0
    markers[unknown == 255] = 0

    # Watershed needs 3-channel image
    wshed_input = cv2.cvtColor(enhanced, cv2.COLOR_GRAY2BGR)
    markers = cv2.watershed(wshed_input, markers)

    # Use watershed boundaries to carve gaps between touching axons
    cleaned = thresh.copy()
    cleaned[markers == -1] = 0  # make watershed boundary black (gap)
    # ---------------------------

    contours, hierarchy = cv2.findContours(
        cleaned, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE
    )
    if hierarchy is None:
        return object_counter
    hierarchy = hierarchy[0]

    for i, contour in enumerate(contours):

        # Only consider outermost contours
        if hierarchy[i][3] != -1:
            continue

        area = cv2.contourArea(contour)
        if area < MIN_CONTOUR_AREA or area > MAX_CONTOUR_AREA:
            continue

        overlap = contour_overlap_ratio(contour, mask_patch)
        if overlap < TISSUE_OVERLAP_MIN:
            continue

        hull = cv2.convexHull(contour)
        solidity = area / (cv2.contourArea(hull) + 1e-6)
        circ = circularity(contour)
        if solidity < MIN_SOLIDITY or circ < MIN_CIRCULARITY:
            continue

        # inner contours
        inner_children = []
        for j, child in enumerate(contours):
            if hierarchy[j][3] == i and cv2.contourArea(child) > 30:
                inner_children.append(child)

        if len(inner_children) == 0:
            continue

        axon_type = "mature" if len(inner_children) <= 2 else "regrowth_cluster"

        (x_outer, y_outer), outer_radius = cv2.minEnclosingCircle(contour)
        contour_shifted = contour + np.array([x_offset, y_offset])
        x_outer += x_offset
        y_outer += y_offset

        outer_area_px = cv2.contourArea(contour)
        outer_area_um2 = outer_area_px * (scale_factor ** 2)

        outer_color = (255, 0, 0) if axon_type == "mature" else (0, 255, 255)
        cv2.drawContours(output_image, [contour_shifted], -1, outer_color, 1)

        inner_radii = []
        inner_areas = []

        for inner_c in inner_children:
            (x_inner, y_inner), inner_radius = cv2.minEnclosingCircle(inner_c)
            inner_shifted = inner_c + np.array([x_offset, y_offset])
            cv2.drawContours(output_image, [inner_shifted], -1, (0, 255, 0), 1)
            inner_radii.append(inner_radius)
            inner_areas.append(cv2.contourArea(inner_c))

        largest_inner = inner_children[np.argmax(inner_areas)]
        inner_radius = max(inner_radii)
        inner_area_px = max(inner_areas)
        inner_area_um2 = inner_area_px * (scale_factor ** 2)

        thickness = (outer_radius - inner_radius) * scale_factor
        g_ratio = inner_radius / (outer_radius + 1e-6)
        area_ratio = inner_area_um2 / (outer_area_um2 + 1e-6)

        axon_data.append({
            "axon_id": object_counter,
            "axon_type": axon_type,
            "num_inner_contours": len(inner_children),
            "center_x": x_outer,
            "center_y": y_outer,
            "outer_radius": outer_radius * scale_factor,
            "inner_radius": inner_radius * scale_factor,
            "thickness": thickness,
            "diameter": (2 * outer_radius) * scale_factor,
            "outer_area": outer_area_um2,
            "inner_area": inner_area_um2,
            "area_ratio": area_ratio,
            "hole_ratio": inner_area_um2 / (outer_area_um2 + 1e-6),
            "eccentricity": 0,
            "g_ratio": g_ratio
        })

        cv2.putText(output_image, str(object_counter),
                    (int(x_outer), int(y_outer)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

        object_counter += 1

    return object_counter

# --------------------------
# Whole image processing
# --------------------------
def process_image(image_path, patch_size=1024):
    image = cv2.imread(image_path)
    if image is None:
        print(f" Cannot read image: {image_path}")
        return None, None

    tissue_mask_full = build_tissue_mask(image)
    h, w = image.shape[:2]

    results = {}

    for mode in ["nodeblur", "deblur"]:
        output_image = image.copy()
        axon_data = []
        object_counter = 1
    
        use_deblur = (mode == "deblur")
    
        for y in range(0, h, patch_size):
            for x in range(0, w, patch_size):
                patch = image[y:y+patch_size, x:x+patch_size]
                mask_patch = tissue_mask_full[y:y+patch_size, x:x+patch_size]
    
                object_counter = process_patch(
                    patch, mask_patch, x, y,
                    axon_data, object_counter,
                    output_image,
                    use_deblur=use_deblur
                )
    
        results[mode] = (axon_data, output_image)
    
    return results


    # for y in range(0, h, patch_size):
    #     for x in range(0, w, patch_size):
    #         patch = image[y:y+patch_size, x:x+patch_size]
    #         mask_patch = tissue_mask_full[y:y+patch_size, x:x+patch_size]

    #         object_counter = process_patch(
    #             patch, mask_patch, x, y, axon_data, object_counter, output_image
    #         )

    # print(f"Processed {len(axon_data)} axons.")
    # return axon_data, output_image

# --------------------------
# Save CSV
# --------------------------
def save_to_csv(axon_data, output_folder):
    df = pd.DataFrame(axon_data)
    csv_path = os.path.join(output_folder, "axon_measurements.csv")
    df.to_csv(csv_path, index=False)
    return df

# --------------------------
# Visualizations
# --------------------------
def plot_and_visualize(df, output_folder):
    numeric_cols = df.select_dtypes(include=[np.number]).columns

    for col in numeric_cols:
        vals = df[col].dropna()
        if len(vals) == 0:
            continue

        plt.hist(vals, bins=25)
        plt.title(col)
        plt.savefig(os.path.join(output_folder, f"hist_{col}.png"))
        plt.close()

# --------------------------
# for viewing effects of blur
# --------------------------
def generate_deblur_preview(image, tissue_mask,
                            kernel_size=7, sigma=2.0, K=0.03):
    """
    Generates full-image Wiener deblur preview (tissue-only)
    """
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Mask background BEFORE deblurring
    gray_masked = gray.copy()
    gray_masked[tissue_mask == 0] = 0

    # Apply Wiener
    deblurred = wiener_deconvolution(
        gray_masked,
        kernel_size=kernel_size,
        sigma=sigma,
        K=K
    )

    return gray, deblurred

# --------------------------
# Process entire folder
# --------------------------
def process_folder(input_folder, output_root):
    images = [
        f for f in os.listdir(input_folder)
        if f.lower().endswith((".png", ".jpg", ".jpeg", ".bmp"))
    ]

    for img_name in images:
        print("\n==============================")
        print(f"Processing {img_name}")
        print("==============================")

        img_path = os.path.join(input_folder, img_name)
        img_name_wo_ext = os.path.splitext(img_name)[0]
        img_output_folder = os.path.join(output_root, img_name_wo_ext)
        os.makedirs(img_output_folder, exist_ok=True)

        # 🔹 Run pipeline ONCE
        results = process_image(img_path)

        # 🔹 Save nodeblur & deblur outputs
        for mode, (axon_data, output_image) in results.items():
            mode_folder = os.path.join(img_output_folder, mode)
            os.makedirs(mode_folder, exist_ok=True)

            df = save_to_csv(axon_data, mode_folder)
            plot_and_visualize(df, mode_folder)

            out_img_path = os.path.join(mode_folder, f"numbered_{img_name}")
            cv2.imwrite(out_img_path, output_image)

        #  Save tissue mask ONCE (shared)
        #image = cv2.imread(img_path)
        #tissue_mask = build_tissue_mask(image)
        # cv2.imwrite(os.path.join(img_output_folder, "tissue_mask.png"), tissue_mask)
        # -------------------------
        # Save deblur preview images
        # -------------------------
        image = cv2.imread(img_path)
        tissue_mask = build_tissue_mask(image)
        
        orig_gray, deblur_gray = generate_deblur_preview(
            image,
            tissue_mask,
            kernel_size=7,
            sigma=2.0,
            K=0.03
        )
        
        # Save originals
        cv2.imwrite(os.path.join(img_output_folder, "input_gray.png"), orig_gray)
        cv2.imwrite(os.path.join(img_output_folder, "deblur_gray.png"), deblur_gray)
        
        # Optional: difference image (what Wiener changed)
        diff = cv2.absdiff(orig_gray, deblur_gray)
        cv2.imwrite(os.path.join(img_output_folder, "deblur_difference.png"), diff)
        
        # Save tissue mask (already doing this)
        cv2.imwrite(os.path.join(img_output_folder, "tissue_mask.png"), tissue_mask)




        # if axon_data is None:
        #     continue

        # df = save_to_csv(axon_data, img_output_folder)
        # plot_and_visualize(df, img_output_folder)
        # image = cv2.imread(img_path)
        # tissue_mask = build_tissue_mask(image)
        
        # mask_path = os.path.join(img_output_folder, "tissue_mask.png")
        # cv2.imwrite(mask_path, tissue_mask)

        # out_img_path = os.path.join(img_output_folder, f"numbered_{img_name}")
        # cv2.imwrite(out_img_path, output_image)

        # print(f"✔ Saved results for {img_name} in {img_output_folder}\n")

def process_multiple_folders(input_root, output_root):
    subfolders = [
        f for f in os.listdir(input_root)
        if os.path.isdir(os.path.join(input_root, f))
    ]

    print(f"Found {len(subfolders)} folders:", subfolders)

    for folder_name in subfolders:
        input_subfolder = os.path.join(input_root, folder_name)
        output_subfolder = os.path.join(output_root, folder_name)

        os.makedirs(output_subfolder, exist_ok=True)

        print("\n====================================")
        print(f"Processing folder: {folder_name}")
        print("====================================")

        process_folder(input_subfolder, output_subfolder)

    print("\n ALL FOLDERS PROCESSED!")


# --------------------------
# RUN
# --------------------------
input_root = r"D:\projects\neuro\all fascicles\fascicle images"
output_root = r"D:\projects\neuro\all fascicles_results_deblur_wiener2"

process_multiple_folders(input_root, output_root)


print("\n DONE! All images processed.")


Found 9 folders: ['set1_Image02_40x_bf_03_fascicle_All', 'set1_image03_40x_bf_12_fascicle_All', 'set1_image04_40x_bf_06_fascicle_All', 'set1_image05_40x_bf_02_all', 'set1_Image_01_40x_bf_02', 'set2_Image_01_20x_bf_02', 'set2_Image_01_20x_bf_05_all', 'set2_Image_02_20x_bf_03', 'set2_image_20x_bf_10_all']

Processing folder: set1_Image02_40x_bf_03_fascicle_All

Processing 1.png

Processing 2.png

Processing 3.png

Processing 4.png

Processing 5.png

Processing 6.png

Processing 7.png

Processing 8.png

Processing 9.png

Processing folder: set1_image03_40x_bf_12_fascicle_All

Processing 1.png

Processing 10.png

Processing 2.png

Processing 3.png

Processing 4.png

Processing 5.png

Processing 6.png

Processing 7.png

Processing 8.png

Processing 9.png

Processing folder: set1_image04_40x_bf_06_fascicle_All

Processing 1.png

Processing 10.png

Processing 11.png

Processing 2.png

Processing 3.png

Processing 4.png

Processing 5.png

Processing 6.png

Processing 7.png

Processing 8.png

P

In [12]:
#lastest code'
import cv2
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# --------------------------
# CONFIG: Give folder containing input images
# --------------------------
#input_folder = r"D:\projects\neuro\all fascicles\set2_Image_01_20x_bf_02"
#output_root = r"D:\projects\neuro\all fascicles\set2_Image_01_20x_bf_02_results3"
#os.makedirs(output_root, exist_ok=True)

# Scale factor (µm/px)
scale_factor = 0.136

# --------------------------
# Tunables (UPDATED)
# --------------------------
TISSUE_OVERLAP_MIN = 0.50
MAX_MEAN_INTENSITY = 245

MIN_CIRCULARITY = 0.05        # lower to detect small axons
MIN_CONTOUR_AREA = 8          # reduced from 20
MAX_CONTOUR_AREA = 2000000
MIN_SOLIDITY = 0.15           # reduced from 0.3

# --------------------------
# Helpers
# --------------------------
def darken_and_sharpen(image):
    darkened = cv2.convertScaleAbs(image, alpha=1.5, beta=-20)
    sharpen_kernel = np.array([[0, -1, 0],
                               [-1, 5, -1],
                               [0, -1, 0]])
    return cv2.filter2D(darkened, -1, sharpen_kernel)

def build_tissue_mask(full_image):
    hsv = cv2.cvtColor(full_image, cv2.COLOR_BGR2HSV)
    H, S, V = cv2.split(hsv)
    sat_mask = cv2.inRange(S, 15, 255)
    not_too_bright = cv2.inRange(V, 0, 250)

    mask = cv2.bitwise_and(sat_mask, not_too_bright)
    k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))

    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, k, iterations=1)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, k, iterations=2)
    return mask

def contour_overlap_ratio(contour, mask):
    c_mask = np.zeros(mask.shape, dtype=np.uint8)
    cv2.drawContours(c_mask, [contour], -1, 255, -1)
    inter = cv2.bitwise_and(c_mask, mask)
    area_c = cv2.countNonZero(c_mask)
    area_i = cv2.countNonZero(inter)
    return area_i / float(area_c) if area_c > 0 else 0.0

def mean_intensity_in_contour(gray, contour):
    m = np.zeros(gray.shape, dtype=np.uint8)
    cv2.drawContours(m, [contour], -1, 255, -1)
    return cv2.mean(gray, mask=m)[0]

def circularity(contour):
    a = cv2.contourArea(contour)
    p = cv2.arcLength(contour, True)
    return 4 * np.pi * a / (p * p + 1e-6) if p > 0 else 0

# --------------------------
# Wiener deconvultion
# --------------------------

def wiener_deconvolution(gray, kernel_size=7, sigma=2.0, K=0.01):
    """
    Safe Wiener deconvolution that adapts PSF to patch size
    """

    h, w = gray.shape

    # If patch is too small, skip deblurring
    if h < kernel_size or w < kernel_size:
        return gray.copy()

    gray_f = gray.astype(np.float32)

    # Ensure kernel size fits patch
    k = min(kernel_size, h, w)
    if k % 2 == 0:
        k -= 1
    k = max(k, 3)

    # Build Gaussian PSF
    psf_1d = cv2.getGaussianKernel(k, sigma)
    psf = psf_1d @ psf_1d.T
    psf /= psf.sum()

    # Pad PSF to image size
    psf_padded = np.zeros_like(gray_f)
    kh, kw = psf.shape
    psf_padded[:kh, :kw] = psf
    psf_padded = np.roll(psf_padded, -kh // 2, axis=0)
    psf_padded = np.roll(psf_padded, -kw // 2, axis=1)

    # FFT
    G = np.fft.fft2(gray_f)
    H = np.fft.fft2(psf_padded)

    # Wiener filter
    H_conj = np.conj(H)
    denom = np.abs(H) ** 2 + K
    F_hat = (H_conj / denom) * G

    # Inverse FFT
    deconv = np.fft.ifft2(F_hat)
    deconv = np.real(deconv)

    # Normalize
    deconv = cv2.normalize(deconv, None, 0, 255, cv2.NORM_MINMAX)
    return deconv.astype(np.uint8)

# --------------------------
# Richardson_lucy
# --------------------------
def richardson_lucy(gray, kernel_size=7, sigma=2.0, iterations=10):
    """
    Safe Richardson–Lucy deconvolution for microscopy
    """

    # Convert to float
    img = gray.astype(np.float32)
    img[img < 0] = 0

    # Build Gaussian PSF
    k = min(kernel_size, img.shape[0], img.shape[1])
    if k % 2 == 0:
        k -= 1
    k = max(k, 3)

    psf_1d = cv2.getGaussianKernel(k, sigma)
    psf = psf_1d @ psf_1d.T
    psf /= psf.sum()

    # Initialize estimate
    estimate = img.copy() + 1e-6

    # RL iterations
    for _ in range(iterations):
        # Convolve estimate with PSF
        conv = cv2.filter2D(estimate, -1, psf)
        conv[conv <= 0] = 1e-6

        # Compute ratio
        relative_blur = img / conv

        # Back-project error
        estimate *= cv2.filter2D(relative_blur, -1, psf[::-1, ::-1])

    # Normalize
    estimate = cv2.normalize(estimate, None, 0, 255, cv2.NORM_MINMAX)
    return estimate.astype(np.uint8)

# --------------------------
# Main patch processing
# --------------------------
def process_patch(patch, mask_patch, x_offset, y_offset,
                  axon_data, object_counter, output_image,
                  use_wiener=False, use_rl=False):

    # Convert to grayscale
    gray = cv2.cvtColor(patch, cv2.COLOR_BGR2GRAY)

    if use_wiener or use_rl:
        masked_gray = gray.copy()
        masked_gray[mask_patch == 0] = 0
    
        if use_wiener:
            masked_gray = wiener_deconvolution(
                masked_gray,
                kernel_size=7,
                sigma=2.0,
                K=0.03
            )
    
        elif use_rl:
            masked_gray = richardson_lucy(
                masked_gray,
                kernel_size=7,
                sigma=2.0,
                iterations=8
            )
    
        gray = masked_gray


    # Convert back to BGR for rest of pipeline
    patch = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)
    patch = darken_and_sharpen(patch)
    gray = cv2.cvtColor(patch, cv2.COLOR_BGR2GRAY)


    # Small smoothing for better boundary separation
    smooth = cv2.bilateralFilter(gray, 5, 20, 20)
    
    # Contrast enhancement (conditional)
    # Contrast enhancement (conditional)
    if use_wiener or use_rl:
        # IMPORTANT: skip CLAHE after any deblurring
        enhanced = smooth
    else:
        clahe = cv2.createCLAHE(clipLimit=2.5, tileGridSize=(8, 8))
        enhanced = clahe.apply(smooth)

    

    # Thresholding
    blurred = cv2.GaussianBlur(enhanced, (3, 3), 0)
    _, otsu = cv2.threshold(blurred, 0, 255,
                            cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    adaptive = cv2.adaptiveThreshold(
        blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY_INV, 35, 2
    )

    thresh = cv2.bitwise_or(otsu, adaptive)

    # ---------------------------
    # NEW: Separate touching axons using WATERSHED
    # ---------------------------
    # thresh: foreground (axon structures) = 255, background = 0
    kernel = np.ones((3, 3), np.uint8)

    # Remove tiny noise, keep main blobs
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=1)

    # Sure background
    sure_bg = cv2.dilate(opening, kernel, iterations=3)

    # Sure foreground via distance transform
    dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
    if dist_transform.max() > 0:
        _, sure_fg = cv2.threshold(dist_transform,
                                   0.4 * dist_transform.max(), 255, 0)
    else:
        sure_fg = np.zeros_like(opening)
    sure_fg = np.uint8(sure_fg)

    # Unknown region (possible boundaries between axons)
    unknown = cv2.subtract(sure_bg, sure_fg)

    # Marker labelling for watershed
    num_markers, markers = cv2.connectedComponents(sure_fg)
    markers = markers + 1  # background becomes 1 instead of 0
    markers[unknown == 255] = 0

    # Watershed needs 3-channel image
    wshed_input = cv2.cvtColor(enhanced, cv2.COLOR_GRAY2BGR)
    markers = cv2.watershed(wshed_input, markers)

    # Use watershed boundaries to carve gaps between touching axons
    cleaned = thresh.copy()
    cleaned[markers == -1] = 0  # make watershed boundary black (gap)
    # ---------------------------

    contours, hierarchy = cv2.findContours(
        cleaned, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE
    )
    if hierarchy is None:
        return object_counter
    hierarchy = hierarchy[0]

    for i, contour in enumerate(contours):

        # Only consider outermost contours
        if hierarchy[i][3] != -1:
            continue

        area = cv2.contourArea(contour)
        if area < MIN_CONTOUR_AREA or area > MAX_CONTOUR_AREA:
            continue

        overlap = contour_overlap_ratio(contour, mask_patch)
        if overlap < TISSUE_OVERLAP_MIN:
            continue

        hull = cv2.convexHull(contour)
        solidity = area / (cv2.contourArea(hull) + 1e-6)
        circ = circularity(contour)
        if solidity < MIN_SOLIDITY or circ < MIN_CIRCULARITY:
            continue

        # inner contours
        inner_children = []
        for j, child in enumerate(contours):
            if hierarchy[j][3] == i and cv2.contourArea(child) > 30:
                inner_children.append(child)

        if len(inner_children) == 0:
            continue

        axon_type = "mature" if len(inner_children) <= 2 else "regrowth_cluster"

        (x_outer, y_outer), outer_radius = cv2.minEnclosingCircle(contour)
        contour_shifted = contour + np.array([x_offset, y_offset])
        x_outer += x_offset
        y_outer += y_offset

        outer_area_px = cv2.contourArea(contour)
        outer_area_um2 = outer_area_px * (scale_factor ** 2)

        outer_color = (255, 0, 0) if axon_type == "mature" else (0, 255, 255)
        cv2.drawContours(output_image, [contour_shifted], -1, outer_color, 1)

        inner_radii = []
        inner_areas = []

        for inner_c in inner_children:
            (x_inner, y_inner), inner_radius = cv2.minEnclosingCircle(inner_c)
            inner_shifted = inner_c + np.array([x_offset, y_offset])
            cv2.drawContours(output_image, [inner_shifted], -1, (0, 255, 0), 1)
            inner_radii.append(inner_radius)
            inner_areas.append(cv2.contourArea(inner_c))

        largest_inner = inner_children[np.argmax(inner_areas)]
        inner_radius = max(inner_radii)
        inner_area_px = max(inner_areas)
        inner_area_um2 = inner_area_px * (scale_factor ** 2)

        thickness = (outer_radius - inner_radius) * scale_factor
        g_ratio = inner_radius / (outer_radius + 1e-6)
        area_ratio = inner_area_um2 / (outer_area_um2 + 1e-6)

        axon_data.append({
            "axon_id": object_counter,
            "axon_type": axon_type,
            "num_inner_contours": len(inner_children),
            "center_x": x_outer,
            "center_y": y_outer,
            "outer_radius": outer_radius * scale_factor,
            "inner_radius": inner_radius * scale_factor,
            "thickness": thickness,
            "diameter": (2 * outer_radius) * scale_factor,
            "outer_area": outer_area_um2,
            "inner_area": inner_area_um2,
            "area_ratio": area_ratio,
            "hole_ratio": inner_area_um2 / (outer_area_um2 + 1e-6),
            "eccentricity": 0,
            "g_ratio": g_ratio
        })

        cv2.putText(output_image, str(object_counter),
                    (int(x_outer), int(y_outer)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

        object_counter += 1

    return object_counter

# --------------------------
# Whole image processing
# --------------------------
def process_image(image_path, patch_size=1024):
    image = cv2.imread(image_path)
    if image is None:
        print(f" Cannot read image: {image_path}")
        return None, None

    tissue_mask_full = build_tissue_mask(image)
    h, w = image.shape[:2]

    results = {}

    for mode in ["nodeblur", "wiener", "rl"]:
        output_image = image.copy()
        axon_data = []
        object_counter = 1
    
        use_wiener = (mode == "wiener")
        use_rl = (mode == "rl")

        for y in range(0, h, patch_size):
            for x in range(0, w, patch_size):
                patch = image[y:y+patch_size, x:x+patch_size]
                mask_patch = tissue_mask_full[y:y+patch_size, x:x+patch_size]
    
                object_counter = process_patch(
                    patch, mask_patch, x, y,
                    axon_data, object_counter,
                    output_image,
                    use_wiener=use_wiener,
                    use_rl=use_rl
                )

    
        results[mode] = (axon_data, output_image)
    
    return results


    # for y in range(0, h, patch_size):
    #     for x in range(0, w, patch_size):
    #         patch = image[y:y+patch_size, x:x+patch_size]
    #         mask_patch = tissue_mask_full[y:y+patch_size, x:x+patch_size]

    #         object_counter = process_patch(
    #             patch, mask_patch, x, y, axon_data, object_counter, output_image
    #         )

    # print(f"Processed {len(axon_data)} axons.")
    # return axon_data, output_image

# --------------------------
# Save CSV
# --------------------------
def save_to_csv(axon_data, output_folder):
    df = pd.DataFrame(axon_data)
    csv_path = os.path.join(output_folder, "axon_measurements.csv")
    df.to_csv(csv_path, index=False)
    return df

# --------------------------
# Visualizations
# --------------------------
def plot_and_visualize(df, output_folder):
    numeric_cols = df.select_dtypes(include=[np.number]).columns

    for col in numeric_cols:
        vals = df[col].dropna()
        if len(vals) == 0:
            continue

        plt.hist(vals, bins=25)
        plt.title(col)
        plt.savefig(os.path.join(output_folder, f"hist_{col}.png"))
        plt.close()

# --------------------------
# for viewing effects of blur
# --------------------------
def generate_deblur_preview(image, tissue_mask,
                            kernel_size=7, sigma=2.0, K=0.03):
    """
    Generates full-image Wiener deblur preview (tissue-only)
    """
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Mask background BEFORE deblurring
    gray_masked = gray.copy()
    gray_masked[tissue_mask == 0] = 0

    # Apply Wiener
    deblurred = wiener_deconvolution(
        gray_masked,
        kernel_size=kernel_size,
        sigma=sigma,
        K=K
    )

    return gray, deblurred

# --------------------------
# Process entire folder
# --------------------------
def process_folder(input_folder, output_root):
    images = [
        f for f in os.listdir(input_folder)
        if f.lower().endswith((".png", ".jpg", ".jpeg", ".bmp"))
    ]

    for img_name in images:
        print("\n==============================")
        print(f"Processing {img_name}")
        print("==============================")

        img_path = os.path.join(input_folder, img_name)
        img_name_wo_ext = os.path.splitext(img_name)[0]
        img_output_folder = os.path.join(output_root, img_name_wo_ext)
        os.makedirs(img_output_folder, exist_ok=True)

        # 🔹 Run pipeline ONCE
        results = process_image(img_path)

        # 🔹 Save nodeblur & deblur outputs
        for mode, (axon_data, output_image) in results.items():
            mode_folder = os.path.join(img_output_folder, mode)
            os.makedirs(mode_folder, exist_ok=True)

            df = save_to_csv(axon_data, mode_folder)
            plot_and_visualize(df, mode_folder)

            out_img_path = os.path.join(mode_folder, f"numbered_{img_name}")
            cv2.imwrite(out_img_path, output_image)

        # 🔹 Save tissue mask ONCE (shared)
        #image = cv2.imread(img_path)
        #tissue_mask = build_tissue_mask(image)
        # cv2.imwrite(os.path.join(img_output_folder, "tissue_mask.png"), tissue_mask)
        # -------------------------
        # Save deblur preview images
        # -------------------------
        image = cv2.imread(img_path)
        tissue_mask = build_tissue_mask(image)
        
        orig_gray, deblur_gray = generate_deblur_preview(
            image,
            tissue_mask,
            kernel_size=7,
            sigma=2.0,
            K=0.03
        )
        
        # Save originals
        cv2.imwrite(os.path.join(img_output_folder, "input_gray.png"), orig_gray)
        cv2.imwrite(os.path.join(img_output_folder, "deblur_gray.png"), deblur_gray)
        
        # Optional: difference image (what Wiener changed)
        diff = cv2.absdiff(orig_gray, deblur_gray)
        cv2.imwrite(os.path.join(img_output_folder, "deblur_difference.png"), diff)
        
        # Save tissue mask (already doing this)
        cv2.imwrite(os.path.join(img_output_folder, "tissue_mask.png"), tissue_mask)




        # if axon_data is None:
        #     continue

        # df = save_to_csv(axon_data, img_output_folder)
        # plot_and_visualize(df, img_output_folder)
        # image = cv2.imread(img_path)
        # tissue_mask = build_tissue_mask(image)
        
        # mask_path = os.path.join(img_output_folder, "tissue_mask.png")
        # cv2.imwrite(mask_path, tissue_mask)

        # out_img_path = os.path.join(img_output_folder, f"numbered_{img_name}")
        # cv2.imwrite(out_img_path, output_image)

        # print(f" Saved results for {img_name} in {img_output_folder}\n")

def process_multiple_folders(input_root, output_root):
    subfolders = [
        f for f in os.listdir(input_root)
        if os.path.isdir(os.path.join(input_root, f))
    ]

    print(f"Found {len(subfolders)} folders:", subfolders)

    for folder_name in subfolders:
        input_subfolder = os.path.join(input_root, folder_name)
        output_subfolder = os.path.join(output_root, folder_name)

        os.makedirs(output_subfolder, exist_ok=True)

        print("\n====================================")
        print(f"Processing folder: {folder_name}")
        print("====================================")

        process_folder(input_subfolder, output_subfolder)

    print("\n ALL FOLDERS PROCESSED!")


# --------------------------
# RUN
# --------------------------
input_root = r"D:\projects\neuro\all fascicles\fascicle images"
output_root = r"D:\projects\neuro\all fascicles_results_deblur_RL"

process_multiple_folders(input_root, output_root)


print("\n DONE! All images processed.")


Found 9 folders: ['set1_Image02_40x_bf_03_fascicle_All', 'set1_image03_40x_bf_12_fascicle_All', 'set1_image04_40x_bf_06_fascicle_All', 'set1_image05_40x_bf_02_all', 'set1_Image_01_40x_bf_02', 'set2_Image_01_20x_bf_02', 'set2_Image_01_20x_bf_05_all', 'set2_Image_02_20x_bf_03', 'set2_image_20x_bf_10_all']

Processing folder: set1_Image02_40x_bf_03_fascicle_All

Processing 1.png

Processing 2.png

Processing 3.png

Processing 4.png

Processing 5.png

Processing 6.png

Processing 7.png

Processing 8.png

Processing 9.png

Processing folder: set1_image03_40x_bf_12_fascicle_All

Processing 1.png

Processing 10.png

Processing 2.png

Processing 3.png

Processing 4.png

Processing 5.png

Processing 6.png

Processing 7.png

Processing 8.png

Processing 9.png

Processing folder: set1_image04_40x_bf_06_fascicle_All

Processing 1.png

Processing 10.png

Processing 11.png

Processing 2.png

Processing 3.png

Processing 4.png

Processing 5.png

Processing 6.png

Processing 7.png

Processing 8.png

P

In [13]:
#lastest code'
import cv2
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# --------------------------
# CONFIG: Give folder containing input images
# --------------------------
#input_folder = r"D:\projects\neuro\all fascicles\set2_Image_01_20x_bf_02"
#output_root = r"D:\projects\neuro\all fascicles\set2_Image_01_20x_bf_02_results3"
#os.makedirs(output_root, exist_ok=True)

# Scale factor (µm/px)
scale_factor = 0.136

# --------------------------
# Tunables (UPDATED)
# --------------------------
TISSUE_OVERLAP_MIN = 0.50
MAX_MEAN_INTENSITY = 245

MIN_CIRCULARITY = 0.05        # lower to detect small axons
MIN_CONTOUR_AREA = 8          # reduced from 20
MAX_CONTOUR_AREA = 2000000
MIN_SOLIDITY = 0.15           # reduced from 0.3

# --------------------------
# Helpers
# --------------------------
def darken_and_sharpen(image):
    darkened = cv2.convertScaleAbs(image, alpha=1.5, beta=-20)
    sharpen_kernel = np.array([[0, -1, 0],
                               [-1, 5, -1],
                               [0, -1, 0]])
    return cv2.filter2D(darkened, -1, sharpen_kernel)

def build_tissue_mask(full_image):
    hsv = cv2.cvtColor(full_image, cv2.COLOR_BGR2HSV)
    H, S, V = cv2.split(hsv)
    sat_mask = cv2.inRange(S, 15, 255)
    not_too_bright = cv2.inRange(V, 0, 250)

    mask = cv2.bitwise_and(sat_mask, not_too_bright)
    k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))

    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, k, iterations=1)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, k, iterations=2)
    return mask

def contour_overlap_ratio(contour, mask):
    c_mask = np.zeros(mask.shape, dtype=np.uint8)
    cv2.drawContours(c_mask, [contour], -1, 255, -1)
    inter = cv2.bitwise_and(c_mask, mask)
    area_c = cv2.countNonZero(c_mask)
    area_i = cv2.countNonZero(inter)
    return area_i / float(area_c) if area_c > 0 else 0.0

def mean_intensity_in_contour(gray, contour):
    m = np.zeros(gray.shape, dtype=np.uint8)
    cv2.drawContours(m, [contour], -1, 255, -1)
    return cv2.mean(gray, mask=m)[0]

def circularity(contour):
    a = cv2.contourArea(contour)
    p = cv2.arcLength(contour, True)
    return 4 * np.pi * a / (p * p + 1e-6) if p > 0 else 0

# --------------------------
# Wiener deconvultion
# --------------------------

def wiener_deconvolution(gray, kernel_size=7, sigma=2.0, K=0.01):
    """
    Safe Wiener deconvolution that adapts PSF to patch size
    """

    h, w = gray.shape

    # If patch is too small, skip deblurring
    if h < kernel_size or w < kernel_size:
        return gray.copy()

    gray_f = gray.astype(np.float32)

    # Ensure kernel size fits patch
    k = min(kernel_size, h, w)
    if k % 2 == 0:
        k -= 1
    k = max(k, 3)

    # Build Gaussian PSF
    psf_1d = cv2.getGaussianKernel(k, sigma)
    psf = psf_1d @ psf_1d.T
    psf /= psf.sum()

    # Pad PSF to image size
    psf_padded = np.zeros_like(gray_f)
    kh, kw = psf.shape
    psf_padded[:kh, :kw] = psf
    psf_padded = np.roll(psf_padded, -kh // 2, axis=0)
    psf_padded = np.roll(psf_padded, -kw // 2, axis=1)

    # FFT
    G = np.fft.fft2(gray_f)
    H = np.fft.fft2(psf_padded)

    # Wiener filter
    H_conj = np.conj(H)
    denom = np.abs(H) ** 2 + K
    F_hat = (H_conj / denom) * G

    # Inverse FFT
    deconv = np.fft.ifft2(F_hat)
    deconv = np.real(deconv)

    # Normalize
    deconv = cv2.normalize(deconv, None, 0, 255, cv2.NORM_MINMAX)
    return deconv.astype(np.uint8)

# --------------------------
# Richardson_lucy
# --------------------------
def richardson_lucy(gray, kernel_size=7, sigma=2.0, iterations=10):
    """
    Safe Richardson–Lucy deconvolution for microscopy
    """

    # Convert to float
    img = gray.astype(np.float32)
    img[img < 0] = 0

    # Build Gaussian PSF
    k = min(kernel_size, img.shape[0], img.shape[1])
    if k % 2 == 0:
        k -= 1
    k = max(k, 3)

    psf_1d = cv2.getGaussianKernel(k, sigma)
    psf = psf_1d @ psf_1d.T
    psf /= psf.sum()

    # Initialize estimate
    estimate = img.copy() + 1e-6

    # RL iterations
    for _ in range(iterations):
        # Convolve estimate with PSF
        conv = cv2.filter2D(estimate, -1, psf)
        conv[conv <= 0] = 1e-6

        # Compute ratio
        relative_blur = img / conv

        # Back-project error
        estimate *= cv2.filter2D(relative_blur, -1, psf[::-1, ::-1])

    # Normalize
    estimate = cv2.normalize(estimate, None, 0, 255, cv2.NORM_MINMAX)
    return estimate.astype(np.uint8)

# --------------------------
# Main patch processing
# --------------------------
def process_patch(patch, mask_patch, x_offset, y_offset,
                  axon_data, object_counter, output_image,
                  use_wiener=False, use_rl=False):

    # Convert to grayscale
    gray = cv2.cvtColor(patch, cv2.COLOR_BGR2GRAY)

    if use_wiener or use_rl:
        masked_gray = gray.copy()
        masked_gray[mask_patch == 0] = 0
    
        if use_wiener:
            masked_gray = wiener_deconvolution(
                masked_gray,
                kernel_size=7,
                sigma=2.0,
                K=0.03
            )
    
        elif use_rl:
            masked_gray = richardson_lucy(
                masked_gray,
                kernel_size=7,
                sigma=2.0,
                iterations=3
            )
    
        gray = masked_gray


    # Convert back to BGR for rest of pipeline
    patch = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)

    # ❗ Sharpen ONLY for nodeblur & wiener
    if not use_rl:
        patch = darken_and_sharpen(patch)
    
    gray = cv2.cvtColor(patch, cv2.COLOR_BGR2GRAY)



    # Small smoothing for better boundary separation
    smooth = cv2.bilateralFilter(gray, 5, 20, 20)
    
    # Contrast enhancement (conditional)
    # Contrast enhancement (conditional)
    if use_wiener or use_rl:
        # IMPORTANT: skip CLAHE after any deblurring
        enhanced = smooth
    else:
        clahe = cv2.createCLAHE(clipLimit=2.5, tileGridSize=(8, 8))
        enhanced = clahe.apply(smooth)

    

    # Thresholding
    blurred = cv2.GaussianBlur(enhanced, (3, 3), 0)
    _, otsu = cv2.threshold(blurred, 0, 255,
                            cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    adaptive = cv2.adaptiveThreshold(
        blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY_INV, 35, 2
    )

    thresh = cv2.bitwise_or(otsu, adaptive)

    # ---------------------------
    # NEW: Separate touching axons using WATERSHED
    # ---------------------------
    # thresh: foreground (axon structures) = 255, background = 0
    kernel = np.ones((3, 3), np.uint8)

    # Remove tiny noise, keep main blobs
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=1)

    # Sure background
    sure_bg = cv2.dilate(opening, kernel, iterations=3)

    # Sure foreground via distance transform
    dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
    if dist_transform.max() > 0:
        _, sure_fg = cv2.threshold(dist_transform,
                                   0.4 * dist_transform.max(), 255, 0)
    else:
        sure_fg = np.zeros_like(opening)
    sure_fg = np.uint8(sure_fg)

    # Unknown region (possible boundaries between axons)
    unknown = cv2.subtract(sure_bg, sure_fg)

    # Marker labelling for watershed
    num_markers, markers = cv2.connectedComponents(sure_fg)
    markers = markers + 1  # background becomes 1 instead of 0
    markers[unknown == 255] = 0

    # Watershed needs 3-channel image
    wshed_input = cv2.cvtColor(enhanced, cv2.COLOR_GRAY2BGR)
    markers = cv2.watershed(wshed_input, markers)

    # Use watershed boundaries to carve gaps between touching axons
    cleaned = thresh.copy()
    cleaned[markers == -1] = 0  # make watershed boundary black (gap)
    # ---------------------------

    contours, hierarchy = cv2.findContours(
        cleaned, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE
    )
    if hierarchy is None:
        return object_counter
    hierarchy = hierarchy[0]

    for i, contour in enumerate(contours):

        # Only consider outermost contours
        if hierarchy[i][3] != -1:
            continue

        area = cv2.contourArea(contour)
        if area < MIN_CONTOUR_AREA or area > MAX_CONTOUR_AREA:
            continue

        overlap = contour_overlap_ratio(contour, mask_patch)
        if overlap < TISSUE_OVERLAP_MIN:
            continue

        hull = cv2.convexHull(contour)
        solidity = area / (cv2.contourArea(hull) + 1e-6)
        circ = circularity(contour)
        if solidity < MIN_SOLIDITY or circ < MIN_CIRCULARITY:
            continue

        # inner contours
        inner_children = []
        for j, child in enumerate(contours):
            if hierarchy[j][3] == i and cv2.contourArea(child) > 30:
                inner_children.append(child)

        if len(inner_children) == 0:
            continue

        axon_type = "mature" if len(inner_children) <= 2 else "regrowth_cluster"

        (x_outer, y_outer), outer_radius = cv2.minEnclosingCircle(contour)
        contour_shifted = contour + np.array([x_offset, y_offset])
        x_outer += x_offset
        y_outer += y_offset

        outer_area_px = cv2.contourArea(contour)
        outer_area_um2 = outer_area_px * (scale_factor ** 2)

        outer_color = (255, 0, 0) if axon_type == "mature" else (0, 255, 255)
        cv2.drawContours(output_image, [contour_shifted], -1, outer_color, 1)

        inner_radii = []
        inner_areas = []

        for inner_c in inner_children:
            (x_inner, y_inner), inner_radius = cv2.minEnclosingCircle(inner_c)
            inner_shifted = inner_c + np.array([x_offset, y_offset])
            cv2.drawContours(output_image, [inner_shifted], -1, (0, 255, 0), 1)
            inner_radii.append(inner_radius)
            inner_areas.append(cv2.contourArea(inner_c))

        largest_inner = inner_children[np.argmax(inner_areas)]
        inner_radius = max(inner_radii)
        inner_area_px = max(inner_areas)
        inner_area_um2 = inner_area_px * (scale_factor ** 2)

        thickness = (outer_radius - inner_radius) * scale_factor
        g_ratio = inner_radius / (outer_radius + 1e-6)
        area_ratio = inner_area_um2 / (outer_area_um2 + 1e-6)

        axon_data.append({
            "axon_id": object_counter,
            "axon_type": axon_type,
            "num_inner_contours": len(inner_children),
            "center_x": x_outer,
            "center_y": y_outer,
            "outer_radius": outer_radius * scale_factor,
            "inner_radius": inner_radius * scale_factor,
            "thickness": thickness,
            "diameter": (2 * outer_radius) * scale_factor,
            "outer_area": outer_area_um2,
            "inner_area": inner_area_um2,
            "area_ratio": area_ratio,
            "hole_ratio": inner_area_um2 / (outer_area_um2 + 1e-6),
            "eccentricity": 0,
            "g_ratio": g_ratio
        })

        cv2.putText(output_image, str(object_counter),
                    (int(x_outer), int(y_outer)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

        object_counter += 1

    return object_counter

# --------------------------
# Whole image processing
# --------------------------
def process_image(image_path, patch_size=1024):
    image = cv2.imread(image_path)
    if image is None:
        print(f" Cannot read image: {image_path}")
        return None, None

    tissue_mask_full = build_tissue_mask(image)
    h, w = image.shape[:2]

    results = {}

    for mode in ["nodeblur", "wiener", "rl"]:
        output_image = image.copy()
        axon_data = []
        object_counter = 1
    
        use_wiener = (mode == "wiener")
        use_rl = (mode == "rl")

        for y in range(0, h, patch_size):
            for x in range(0, w, patch_size):
                patch = image[y:y+patch_size, x:x+patch_size]
                mask_patch = tissue_mask_full[y:y+patch_size, x:x+patch_size]
    
                object_counter = process_patch(
                    patch, mask_patch, x, y,
                    axon_data, object_counter,
                    output_image,
                    use_wiener=use_wiener,
                    use_rl=use_rl
                )

    
        results[mode] = (axon_data, output_image)
    
    return results


    # for y in range(0, h, patch_size):
    #     for x in range(0, w, patch_size):
    #         patch = image[y:y+patch_size, x:x+patch_size]
    #         mask_patch = tissue_mask_full[y:y+patch_size, x:x+patch_size]

    #         object_counter = process_patch(
    #             patch, mask_patch, x, y, axon_data, object_counter, output_image
    #         )

    # print(f"Processed {len(axon_data)} axons.")
    # return axon_data, output_image

# --------------------------
# Save CSV
# --------------------------
def save_to_csv(axon_data, output_folder):
    df = pd.DataFrame(axon_data)
    csv_path = os.path.join(output_folder, "axon_measurements.csv")
    df.to_csv(csv_path, index=False)
    return df

# --------------------------
# Visualizations
# --------------------------
def plot_and_visualize(df, output_folder):
    numeric_cols = df.select_dtypes(include=[np.number]).columns

    for col in numeric_cols:
        vals = df[col].dropna()
        if len(vals) == 0:
            continue

        plt.hist(vals, bins=25)
        plt.title(col)
        plt.savefig(os.path.join(output_folder, f"hist_{col}.png"))
        plt.close()

# --------------------------
# for viewing effects of blur
# --------------------------
def generate_deblur_preview(image, tissue_mask,
                            kernel_size=7, sigma=2.0, K=0.03):
    """
    Generates full-image Wiener deblur preview (tissue-only)
    """
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Mask background BEFORE deblurring
    gray_masked = gray.copy()
    gray_masked[tissue_mask == 0] = 0

    # Apply Wiener
    deblurred = wiener_deconvolution(
        gray_masked,
        kernel_size=kernel_size,
        sigma=sigma,
        K=K
    )

    return gray, deblurred

# --------------------------
# Process entire folder
# --------------------------
def process_folder(input_folder, output_root):
    images = [
        f for f in os.listdir(input_folder)
        if f.lower().endswith((".png", ".jpg", ".jpeg", ".bmp"))
    ]

    for img_name in images:
        print("\n==============================")
        print(f"Processing {img_name}")
        print("==============================")

        img_path = os.path.join(input_folder, img_name)
        img_name_wo_ext = os.path.splitext(img_name)[0]
        img_output_folder = os.path.join(output_root, img_name_wo_ext)
        os.makedirs(img_output_folder, exist_ok=True)

        # 🔹 Run pipeline ONCE
        results = process_image(img_path)

        # 🔹 Save nodeblur & deblur outputs
        for mode, (axon_data, output_image) in results.items():
            mode_folder = os.path.join(img_output_folder, mode)
            os.makedirs(mode_folder, exist_ok=True)

            df = save_to_csv(axon_data, mode_folder)
            plot_and_visualize(df, mode_folder)

            out_img_path = os.path.join(mode_folder, f"numbered_{img_name}")
            cv2.imwrite(out_img_path, output_image)

        # 🔹 Save tissue mask ONCE (shared)
        #image = cv2.imread(img_path)
        #tissue_mask = build_tissue_mask(image)
        # cv2.imwrite(os.path.join(img_output_folder, "tissue_mask.png"), tissue_mask)
        # -------------------------
        # Save deblur preview images
        # -------------------------
        image = cv2.imread(img_path)
        tissue_mask = build_tissue_mask(image)
        
        orig_gray, deblur_gray = generate_deblur_preview(
            image,
            tissue_mask,
            kernel_size=7,
            sigma=2.0,
            K=0.03
        )
        
        # Save originals
        cv2.imwrite(os.path.join(img_output_folder, "input_gray.png"), orig_gray)
        cv2.imwrite(os.path.join(img_output_folder, "deblur_gray.png"), deblur_gray)
        
        # Optional: difference image (what Wiener changed)
        diff = cv2.absdiff(orig_gray, deblur_gray)
        cv2.imwrite(os.path.join(img_output_folder, "deblur_difference.png"), diff)
        
        # Save tissue mask (already doing this)
        cv2.imwrite(os.path.join(img_output_folder, "tissue_mask.png"), tissue_mask)




        # if axon_data is None:
        #     continue

        # df = save_to_csv(axon_data, img_output_folder)
        # plot_and_visualize(df, img_output_folder)
        # image = cv2.imread(img_path)
        # tissue_mask = build_tissue_mask(image)
        
        # mask_path = os.path.join(img_output_folder, "tissue_mask.png")
        # cv2.imwrite(mask_path, tissue_mask)

        # out_img_path = os.path.join(img_output_folder, f"numbered_{img_name}")
        # cv2.imwrite(out_img_path, output_image)

        # print(f" Saved results for {img_name} in {img_output_folder}\n")

def process_multiple_folders(input_root, output_root):
    subfolders = [
        f for f in os.listdir(input_root)
        if os.path.isdir(os.path.join(input_root, f))
    ]

    print(f"Found {len(subfolders)} folders:", subfolders)

    for folder_name in subfolders:
        input_subfolder = os.path.join(input_root, folder_name)
        output_subfolder = os.path.join(output_root, folder_name)

        os.makedirs(output_subfolder, exist_ok=True)

        print("\n====================================")
        print(f"Processing folder: {folder_name}")
        print("====================================")

        process_folder(input_subfolder, output_subfolder)

    print("\n ALL FOLDERS PROCESSED!")


# --------------------------
# RUN
# --------------------------
input_root = r"D:\projects\neuro\all fascicles\fascicle images"
output_root = r"D:\projects\neuro\all fascicles_results_deblur_RL2"

process_multiple_folders(input_root, output_root)


print("\n DONE! All images processed.")


Found 9 folders: ['set1_Image02_40x_bf_03_fascicle_All', 'set1_image03_40x_bf_12_fascicle_All', 'set1_image04_40x_bf_06_fascicle_All', 'set1_image05_40x_bf_02_all', 'set1_Image_01_40x_bf_02', 'set2_Image_01_20x_bf_02', 'set2_Image_01_20x_bf_05_all', 'set2_Image_02_20x_bf_03', 'set2_image_20x_bf_10_all']

Processing folder: set1_Image02_40x_bf_03_fascicle_All

Processing 1.png

Processing 2.png

Processing 3.png

Processing 4.png

Processing 5.png

Processing 6.png

Processing 7.png

Processing 8.png

Processing 9.png

Processing folder: set1_image03_40x_bf_12_fascicle_All

Processing 1.png

Processing 10.png

Processing 2.png

Processing 3.png

Processing 4.png

Processing 5.png

Processing 6.png

Processing 7.png

Processing 8.png

Processing 9.png

Processing folder: set1_image04_40x_bf_06_fascicle_All

Processing 1.png

Processing 10.png

Processing 11.png

Processing 2.png

Processing 3.png

Processing 4.png

Processing 5.png

Processing 6.png

Processing 7.png

Processing 8.png

P